In [1]:
from datasets import Dataset
from datasets import load_dataset
from glob import glob
import pandas as pd
import numpy as np

In [2]:
sdxl_1step = pd.read_csv("data/a100/diffusion/batchsize_1_output_ncu.csv")
metrics = sdxl_1step["Metric Name"]
ind = np.where(metrics == "Duration")[0]
kernel = sdxl_1step.iloc[ind]["Kernel Name"]
duration = sdxl_1step.iloc[ind]["Metric Value"].str.replace(',', '').astype(float)/1e9
simplified = pd.DataFrame({
    'Kernel': kernel,
    'Duration': duration
})
total_duration= simplified.groupby('Kernel')['Duration'].sum()
sorted_kernel_durations = total_duration.sort_values(ascending=False)

In [3]:
sum(duration)

0.3821615040000006

In [4]:
sorted_kernel_durations

Kernel
void at::native::<unnamed>::RowwiseMomentsCUDAKernel<float>(long, T1, const T1 *, T1 *, T1 *)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       0.073837
sm80_xmma_fprop_implicit_gemm_tf32f32_tf32f32_f32_nhwckrsc_nchw_tilesize256x128x32_stage3_warpsize4x2x1_g1_tensor16x8x8_execute_kernel_cudnn_infer                                                                                                                  